In [1]:
from plugins.environments.awa_environment import AWAEnvironment
from plugins.interfaces.awa_interface import AWAInterface

# import data from csv file
import pandas as pd
variable_file = "plugins/environments/awa_variables.csv"
observable_file = "plugins/environments/awa_observables.csv"

env = AWAEnvironment(variable_file, observable_file, interface=AWAInterface(), target_charge=1.0)

In [2]:
env.variables

{'AWA:Drive:DS1:Ctrl': [500.0, 550.0],
 'AWA:Drive:DS3:Ctrl': [180.0, 260.0],
 'AWA:Bira3Ctrl:Ch03': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch04': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch05': [-5.0, 5.0],
 'AWA:Bira3Ctrl:Ch06': [-5.0, 5.0],
 'AWALLRF:K1:SetPhase': [236.0, 256.0],
 'AWA:DAC0:Ch08': [4.5, 5.9]}

In [3]:
from xopt import Xopt, Evaluator, VOCS
from xopt.generators.bayesian import BayesianExplorationGenerator, UpperConfidenceBoundGenerator
from xopt.generators.bayesian.models.standard import StandardModelConstructor
from xopt.generators.bayesian.turbo import SafetyTurboController, OptimizeTurboController

import time

def evaluate(inputs):
    env.set_variables(inputs)
    time.sleep(2.0)
    result = env.get_observables(["13ARV1:Sx"])
    result["total_rms_size"] = (result["13ARV1:Sx"]**2 + result["13ARV1:Sy"]**2)**0.5
    return result

# use only one variable
vocs = VOCS(variables=env.variables,
            objectives={"total_rms_size":"MINIMIZE"},
            constraints={"13ARV1:penalty":["LESS_THAN",0.0]})


In [4]:

model_constructor = StandardModelConstructor(use_low_noise_prior=False)
generator = BayesianExplorationGenerator(
    vocs=vocs, 
    model_constructor=model_constructor,
    turbo_controller=SafetyTurboController(vocs=vocs, length=0.1)
)
evaluator = Evaluator(function=evaluate)
X = Xopt(vocs=vocs, evaluator=evaluator, generator=generator)
X.options.dump_file = "exploration_2_nd_filter.yml"

In [5]:
X


            Xopt
________________________________
Version: 1.4.1+50.ge9fc8ac.dirty
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: false, dump_file: exploration_2_nd_filter.yml, max_evaluations: null}
generator:
  name: bayesian_exploration
  model: null
  turbo_controller:
    dim: 8
    batch_size: 1
    length: 0.1
    length_min: 0.0078125
    length_max: 2.0
    failure_counter: 0
    failure_tolerance: 4
    success_counter: 0
    success_tolerance: 4
    center_x: null
    scale_factor: 1.25
    tkwargs: {dtype: torch.float64}
  use_cuda: false
  model_constructor:
    name: standard
    use_low_noise_prior: false
    covar_modules: {}
    mean_modules: {}
    trainable_mean_keys: []
    dtype: torch.float64
    device: cpu
  numerical_optimizer: {name: LBFGS, n_raw_samples: 20, n_restarts: 20, max_iter: 2000}
  max_travel_distances: null
  n_monte_carlo_samples: 128
evaluator:
  function: __main__.evaluate
  max_workers: 1
  function_kwargs: {}
  vectorized: false
v

In [6]:
# get the current quad setpoint
#current_val = env.get_variables(['AWA:Bira3Ctrl:Ch03'])
import numpy as np
default_pt = np.array([0, 0, 0, 0,5.9,550,190,246])
default_val = dict(zip(X.vocs.variable_names, default_pt))
default_val = pd.DataFrame(default_val, index=[0])
default_val

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase
0,0.0,0.0,0.0,0.0,5.9,550.0,190.0,246.0


In [7]:
# evaluate that point in xopt
X.evaluate_data(default_val)

CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArraySize1_RBV", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Thu Jun 15 2023 15:04:23.649051673
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArrayData", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Thu Jun 15 2023 15:04:23.649200834
..................................................................
CA.Client.Exception...............................................
    Context: "Channel: "13ARV1:image1:ArraySize0_RBV", Connecting to: 146.139.52.185:5064, Ignored: awa5:5064"
    Source File: ../cac.cpp line 1320
    Current Time: Thu Jun 15 2023 15:04:23.649287060
................................................................

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,13ARV1:image1:ArraySize1_RBV,AWAVXI11ICT:Ch1,...,AWAVXI11ICT:Ch1_std,13ARV1:image1:ArraySize0_RBV_std,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:penalty_std,total_rms_size,xopt_runtime,xopt_error
1,0.0,0.0,0.0,0.0,5.9,550.0,190.0,246.0,1200.0,9.637042e-10,...,5.452919e-11,0.0,1.042786,1.078548,2.934228,1.814375,6.755243,57.000278,6.519966,False


In [8]:
# evaluate a second nearby point
second_pt = np.array([0.1, 0.1, 0.1, 0.1, 5.8,540,185,245])
second_pt = dict(zip(X.vocs.variable_names, second_pt))
second_pt = pd.DataFrame(second_pt, index=[0])
X.evaluate_data(second_pt)

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,13ARV1:image1:ArraySize1_RBV,AWAVXI11ICT:Ch1,...,AWAVXI11ICT:Ch1_std,13ARV1:image1:ArraySize0_RBV_std,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:penalty_std,total_rms_size,xopt_runtime,xopt_error
2,0.1,0.1,0.1,0.1,5.8,540.0,185.0,245.0,1200.0,9.656808e-10,...,3.334863e-11,0.0,3.716726,0.674299,5.276837,0.253519,8.346779,105.464284,7.477461,False


In [9]:
X.data

,AWA:Bira3Ctrl:Ch03,AWA:Bira3Ctrl:Ch04,AWA:Bira3Ctrl:Ch05,AWA:Bira3Ctrl:Ch06,AWA:DAC0:Ch08,AWA:Drive:DS1:Ctrl,AWA:Drive:DS3:Ctrl,AWALLRF:K1:SetPhase,13ARV1:image1:ArraySize1_RBV,AWAVXI11ICT:Ch1,...,AWAVXI11ICT:Ch1_std,13ARV1:image1:ArraySize0_RBV_std,13ARV1:Cx_std,13ARV1:Cy_std,13ARV1:Sx_std,13ARV1:Sy_std,13ARV1:penalty_std,total_rms_size,xopt_runtime,xopt_error
1,0.0,0.0,0.0,0.0,5.9,550.0,190.0,246.0,1200.0,9.637042e-10,...,5.452919e-11,0.0,1.042786,1.078548,2.934228,1.814375,6.755243,57.000278,6.519966,False
2,0.1,0.1,0.1,0.1,5.8,540.0,185.0,245.0,1200.0,9.656808e-10,...,3.334863e-11,0.0,3.716726,0.674299,5.276837,0.253519,8.346779,105.464284,7.477461,False


In [ ]:
# run exploration
n_steps = 200
X.generator.numerical_optimizer.max_iter = 50
for i in range(n_steps):
    print(i)
    start = time.time()
    X.step()
    print(time.time() - start)
    

0
9.095255851745605
1
8.19329309463501
2
10.320689916610718
3
13.00560975074768
4
11.943837642669678
5
9.826449155807495
6


/home/awa/miniconda3/envs/awa-badger/lib/python3.9/site-packages/botorch/optim/optimize.py:366: RuntimeWarning: Optimization failed in `gen_candidates_scipy` with the following warning(s):
[OptimizationWarning('Optimization failed within `scipy.optimize.minimize` with status 2 and message ABNORMAL_TERMINATION_IN_LNSRCH.')]
Trying again with a new set of initial conditions.
  warnings.warn(first_warn_msg, RuntimeWarning)


13.45125126838684
7
9.964389324188232
8
15.270519495010376
9


In [ ]:
X.data

In [ ]:
X.data.plot(y=X.vocs.variable_names[:4])

In [ ]:
X.data.plot(y=X.vocs.variable_names[4])

In [ ]:
X.data.plot(y=X.vocs.variable_names[5:])

In [ ]:
import torch
import matplotlib.pyplot as plt
test_x = torch.linspace(-2,2, 100)
model = X.generator.train_model()

fig,ax = plt.subplots(2,1, sharex="all")
fig.set_size_inches(6,6)
with torch.no_grad():
    post = model.posterior(test_x.reshape(-1,1,1).double())
    for i in range(post.event_shape[-1]):
        mean = post.mean[...,i].squeeze()
        l,u = post.mvn.confidence_region()
        ax[0].plot(test_x, mean,f"C{i}", label=generator.vocs.output_names[i])
        ax[0].fill_between(test_x, l[...,i].squeeze(), u[...,i].squeeze(), alpha=0.5)


    acq = generator.get_acquisition(model)(test_x.reshape(-1,1,1).double())

    ax[1].plot(test_x, acq, label='Acquisition Function')
    ax[1].legend()

In [ ]:
from emitopt.utils import get_valid_emittance_samples
beam_energy = 45*10**-3 # GeV 
q_len = 0.12 # m
distance = 1.33-0.265 #m

data = X.data

data["grad"] = data["AWA:Bira3Ctrl:Ch04"] * 100*8.93e-3
data["int_grad"] = data["grad"]*q_len*10

data["sx_m"] = 3.9232781168265036e-05 * data["13ARV1:Sx"]

data


In [ ]:
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.priors import GammaPrior
from gpytorch.kernels import MaternKernel, PolynomialKernel, ScaleKernel
from gpytorch import ExactMarginalLogLikelihood

from botorch.models.gp_regression import SingleTaskGP
from botorch.models.transforms import Normalize, Standardize
from botorch.fit import fit_gpytorch_mll

train_x = torch.tensor(data.dropna()["int_grad"].to_numpy()).double().unsqueeze(1)
train_y = torch.tensor(data.dropna()["sx_m"].to_numpy()).double().unsqueeze(1)

print(train_x.shape)
print(train_y.shape)
input_transform = Normalize(1)
outcome_transform = Standardize(1)
covar_module = ScaleKernel(PolynomialKernel(power=2))
#covar_module = MaternKernel()

model = SingleTaskGP(train_x, 
                     train_y, 
                     input_transform=input_transform,
                     outcome_transform=outcome_transform, 
                     covar_module = covar_module
                     )

mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll)


(emits_at_target_valid,
 emits_sq_at_target,
 is_valid,
 sample_validity_rate) = get_valid_emittance_samples(model, beam_energy,
                                                     q_len,
                                                     distance, n_samples=50, n_steps_quad_scan=10, visualize=True)

In [ ]:
plt.hist(emits_at_target_valid.flatten()*90)
plt.title('Distribution of Sampled Emittances')
plt.xlabel('Emittance')
plt.ylabel('Probability Density')